In [ ]:
#IMPORTS
from kaggle_secrets import UserSecretsClient
from geopy.geocoders import Nominatim
from datetime import date, timedelta
from tqdm import tqdm
import pandas as pd
import numpy as np
import requests
import re

In [ ]:
#API KEY
user_secrets = UserSecretsClient()
aqicn_key = user_secrets.get_secret("aqicn_token")
telraam_key = user_secrets.get_secret("telraam_key")
tomtom_key = user_secrets.get_secret("tomtom_key")

In [ ]:
url = f'https://api.tomtom.com/traffic/trafficstats/trafficdensity/1?key={tomtom_key}'

payload = {
  "jobName":"Test job",
  "distanceUnit":"KILOMETERS",
  "network": {
    "name": "test",
      "geometry" : {
      "type": "MultiPolygon",
      "coordinates": [
        [
          [
            [19.44305, 51.75612],
            [19.44992, 51.75612],
            [19.44992, 51.75947],
            [19.44305, 51.75947],
            [19.44305, 51.75612]
          ]
        ],
        [
          [
            [19.45011, 51.75789],
            [19.45687, 51.75789],
            [19.45687, 51.75946],
            [19.45011, 51.75946],
            [19.45011, 51.75789]
          ]
        ]
      ]
    },
    "timeZoneId": "Europe/Warsaw",
    "frcs": [
      "0",
      "1",
      "2",
      "3",
      "4",
      "5",
      "6",
      "7",
      "8"
    ]
},
  "dateRange":
    {
      "name":"Last working week of January",
      "from":"2021-01-25",
      "to":"2021-01-29",
      "exclusions":[
        "2021-01-26",
        "2021-01-27"
      ]
    },
  "timeSets":[
    {
      "name":"Monday morning hour",
      "timeGroups":[
        {
          "days":[
            "MON"
          ],
          "times":[
            "7:00-8:00"
          ]
        }
      ]
    }
  ]
}

response = requests.request("POST", url, data=payload)
response.content

In [ ]:
#TELRAAM
streets = {
    'AbramJoffe_straße': 9000002109,
    'AlteJakob_straße': 9000002582,
    'Blumenthal_straße': 9000004110,
    'Böckh_straße': 9000002445,
    'Dörpfeldstrasse': 9000002074,
    'Emser_straße': 9000002685,
    'Friesen_straße': 9000003088,
    'Fanninger_straße': 9000003271,
    'Friedel_straße': 9000004118,
    'Gemeinschafts_straße': 9000002204,
    'Genossenschafts_straße': 9000003642,
    'Gleim_straße': 9000003874,
    'Galenus_straße': 9000004233,
    'Grenz_straße': 9000003520,
    'Hentig_straße': 9000003620,
    'Heiligenberger_straße': 9000004336,
    'Heidelberger_straße': 9000004336,
    'Hertzberg_straße': 9000002889,
    'Jessner_straße': 9000004132,
    'Karlsgarten_straße': 9000003783,
    'Lück_straße': 9000003747,
    'Leonhardyweg_straße': 9000003334,
    'Langenscheidt_straße': 9000003997,
    'Lück_straße': 9000003740,
    'Gutenberg_straße': 9000003306,
    'Rolandseck_straße': 9000003991,
    'Radicke_straße': 9000003318,
    'Rheinstein_straße': 9000004284,
    'Tegeler_straße': 9000003144,
    'Matthias_straße': 9000003702,
    'Niederbarnim_straße': 9000003000,
    'Platanen_straße': 9000003006,
    'Proskauer_straße': 9000003072,
    'Rathaus_straße': 9000003738,
    'Schott_straße': 9000004116,
    'Schönlein_straße': 9000004089,
    'Seelower_straße': 9000003786,
    'Simplon_straße': 9000003076,
    'SalvadorAllende-straße': 9000003274,
    'Tuchoslky_straße': 9000004033,
    'Uhland_straße': 9000003439,
    'Vineta_straße': 9000003731,
    'Waldenser_straße': 9000003760,
    'Waldowallee_straße': 9000003172,
    'WernerVossDamm_straße': 9000003312,
    'Weimarer_straße': 9000003267,
    'Wühlisch_straße': 9000004286,
    'Württembergische_straße': 9000003561,
    'Wilski_straße': 9000003084,
    'Xantener_straße': 9000004035,
    'Zossener_Straße': 9000003905,
}

In [ ]:
#LIVE DATA FROM ALL CAMERAS  
url = "https://telraam-api.net/v1/reports/traffic_snapshot_live"

payload = {}

headers = {
  'X-Api-Key': telraam_key
}

response = requests.request("GET", url, headers=headers, data=payload)
response.content
json = response.json()
data = pd.DataFrame(json['features'])

In [ ]:
#ALL TELRAAM DEVICES FROM BERLIN AREA
url = "https://telraam-api.net/v1/reports/traffic_snapshot"

payload = {
    'time':'live',
    'contents':'minimal',
    'area':'13.115715454463032, 52.67212298591935, 13.735069652205851, 52.33524268936387'
    }

headers = {
  'X-Api-Key': telraam_key
}

response = requests.request("POST", url, headers=headers, data=str(payload))
json = response.json()
telraam_data = pd.DataFrame(json['features'])

#SETTING COORDINATES
coordinates = [str(point['coordinates'][0][len(point['coordinates'])//2]) for point in telraam_data['geometry']]
telraam_data.drop(['type', 'geometry'], axis=1, inplace=True)
telraam_data['coordinates'] = coordinates
telraam_data.set_index('coordinates', inplace=True)

#SETTING VALUES
values = [list(all_values.values()) for all_values in telraam_data['properties']]
telraam_data[list(telraam_data['properties'][0].keys())] = values
telraam_data.drop('properties', axis=1, inplace=True)

In [ ]:
segments = list(telraam_data['segment_id'])
coordinates = list(telraam_data.index)
segment_coord = dict(zip(segments, coordinates))
test_segment_coord = dict(list(segment_coord.items())[:5])

In [ ]:
#CHECK WHICH SEGMENTS ARE REGULARY ONLINE
online_stations = telraam_data.loc[telraam_data['uptime'].str.isalpha() != False]

In [ ]:
#FETCH PAST DATA FROM ALL DEVICES - TELRAAM
telraam_history = pd.DataFrame()

url = "https://telraam-api.net/v1/reports/traffic"

today = date.today().strftime("%Y-%m-%d 00:00:00")
last_year = (date.today() - timedelta(days = 90)).strftime("%Y-%m-%d 00:00:00")

for segment, coordinate in test_segment_coord.items():
    payload  = {
    "level": "segments",
    "format": "per-hour",
    "id": segment,
    "time_start": last_year,
    "time_end": today
    }

    headers = {
      'X-Api-Key': telraam_key
    }

    response = requests.request("POST", url, headers=headers, data=str(payload))
    json = response.json()
    df_temp = pd.DataFrame(json['report'])
    df_temp['coordinates'] = coordinate
    telraam_history = pd.concat([telraam_history, df_temp[['segment_id', 'date', 'heavy', 'car', 'coordinates']]])
    
telraam_history    

In [ ]:
# COMMUNITY SENSOR DATA
lat_lang = '52.67212298591935,13.115715454463032,52.33524268936387,13.735069652205851'
url = f'https://data.sensor.community/airrohr/v1/filter/box={lat_lang}&type=SDS011'
data = requests.get(url)
json = data.json()
data = pd.DataFrame(json)

In [ ]:
#FETCHING ALL DATA FROM COMMUNITY TABLE
url = f'https://public.opendatasoft.com/api/records/1.0/search/?dataset=api-luftdateninfo&q=Berlin&rows=1000&facet=timestamp&facet=land&facet=value_type&facet=is_indoor&refine.is_indoor=0'
data = requests.get(url)
json = data.json()
df = pd.DataFrame(json['records'])
df = pd.DataFrame(list(df['fields']))
df

In [ ]:
#ONLY 18 STATIONS
lat_lang = '52.67212298591935,13.115715454463032,52.33524268936387,13.735069652205851'
url = f'https://api.waqi.info/v2/map/bounds?latlng={lat_lang}&networks=all&token={aqicn_key}'
stations = requests.get(url)
json = stations.json()
df = pd.DataFrame(json)
df = pd.DataFrame(list(df['data']))

In [ ]:
#BERLIN AIR QUALITY AQICN
lat = 52.5088793031756
lng = 13.3095203936159
url = f'https://api.waqi.info/feed/geo:{lat};{lng}/?token={aqicn_key}'
air_quality = requests.get(url)
json = air_quality.json()
df = pd.DataFrame(json)
df

In [ ]:
#BERLIN AIR QUALITY 
station_link = 'https://luftdaten.berlin.de/api/stations?active=true&include_hidden=false'
station_request = requests.get(station_link)
all_stations = pd.read_json(station_request.content)
stations_code = all_stations['code']

data_link = 'https://luftdaten.berlin.de/api/stations/mc010/data?period=1h&timespan=custom&start=01.11.2021%2000%3A00&end=01.11.2022%2000%3A00'
data_request = requests.get(data_link)
data = pd.read_json(data_request.content)

In [ ]:
all_stations

In [ ]:
all_stations.apply(lambda x: x.lat + x.lng)

In [ ]:
lowest_distane = []
for coord in online_stations.index:
    coord = re.sub("[^0-9 .]", '', coord)
    lng, lat = coord.split()
    coord = [float(lat), float(lng)]
    smallest = map(lambda x: x-)
    break

In [ ]:
#TOM TOM - 2500 querries na dzień

link = f'https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?key={tomtom_key}&point=52.524934,13.410133'
traffic_request = requests.get(link)
print(traffic_request.status_code)
traffic_data = pd.read_json(traffic_request.content)
traffic_flow = np.nan
if traffic_data.loc['roadClosure'][0] == False
    traffic_flow = float(traffic_data.loc['freeFlowSpeed'] * 100 / traffic_data.loc['freeFlowTravelTime'])

In [ ]:
#FIND NAME OF THE STREET
locator = Nominatim(user_agent='myGeocoder')
coordinates = '52.5090895031756, 13.3094958936159'
location = locator.reverse(coordinates)
location.raw['address']['road']

In [ ]:
#UK
GROUPS = 'https://api.erg.ic.ac.uk/AirQuality/Information/Groups/Json' 
GROUPS = requests.get(GROUPS)

SITES = 'https://api.erg.ic.ac.uk/AirQuality/Information/MonitoringSites/GroupName=London'
SITES = requests.get(SITES)

In [ ]:
sites = pd.read_xml(SITES.content)
open_sites = sites[sites['DateClosed'].isna()]
site_codes = list(open_sites['SiteCode'])

In [ ]:
#UK AIR QUALITY

df = pd.DataFrame()
today = date.today().strftime("%Y-%m-%d")
yesterday = (date.today() - timedelta(days = 1)).strftime("%Y-%m-%d")

for i in tqdm(range(len(site_codes))):
    try:
        querry = f'https://api.erg.ic.ac.uk/AirQuality/Data/Site/SiteCode={site_codes[i]}/StartDate={yesterday}/EndDate={today}'
        data = requests.get(querry)
        temp_df = pd.read_xml(data.content)
        temp_df = temp_df.pivot_table(index=['MeasurementDateGMT'], columns=['SpeciesCode'], values="Value")
        temp_df.insert(0, 'SITE_CODE', site_codes[i])
        df = pd.concat([df, temp_df])
    except:
        print(site_codes[i])
        continue

In [ ]:
#UK AIR QUALITY

all_sites = list(df['SITE_CODE'].unique())
check_sites = list(set(site_codes) - set(all_sites))

errors = pd.DataFrame()
today = date.today().strftime("%Y-%m-%d")
yesterday = (date.today() - timedelta(days = 1)).strftime("%Y-%m-%d")

for site in check_sites:
        querry = f'https://api.erg.ic.ac.uk/AirQuality/Data/Site/SiteCode={site}/StartDate={yesterday}/EndDate={today}'
        print(querry)
        data = requests.get(querry)
        print(pd.read_xml(data.content))

In [ ]:
#SEOUL POLLUTION DATA

CO2_PATH = "../input/seoul-co2/Seoul co2.csv"
POLLUTION_PATH = "../input/air-pollution-in-seoul/AirPollutionSeoul/Measurement_summary.csv"

co2_data = pd.read_csv(CO2_PATH).drop('City', axis=1)
co2_data['Date'] = pd.date_range(start="2019-01-01",end="2019-12-31").date
co2_data['Date'] = co2_data['Date'].astype(str)
co2_data.set_index('Date', inplace=True)

pollution_data = pd.read_csv(POLLUTION_PATH).drop(['Station code', 'Address', 'Latitude', 'Longitude'], axis=1)
pollution_data = pollution_data[pollution_data['Measurement date'].str.contains('2019')].set_index("Measurement date")
pollution_data = pollution_data[pollution_data > 0]

In [ ]:
nan = pollution_data.isna().sum().sum()
notnan = pollution_data.notna().sum().sum()
print(f'NAN Percentage {round(nan*100/notnan, 2)}%')

In [ ]:
dates = pd.date_range(start="2019-01-01",end="2019-12-31")
values = []
for date in dates:
    day = str(date.date())
    df_temp = pollution_data[pollution_data.index.str.contains(day)]
    value = df_temp.mean().values
    values.append([day, *value])

df = pd.DataFrame(values, columns=pollution_data.columns.insert(0, 'DATE'))
df.dropna(inplace=True)
df.set_index('DATE', inplace=True)
df.sort_index(inplace=True)
df = df.merge(co2_data, left_index=True, right_index=True)